Modified: Jul 17, 2019
# Curve Representation using Levelset function
- Signed Distance Function


In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, time
import numpy as np
import scipy as sp
from scipy.signal import correlate2d
import pandas as pd
    
from pathlib import Path
from pprint import pprint as pp

from sklearn.externals import joblib
import pdb

import matplotlib.pyplot as plt
%matplotlib inline

# ignore warnings
import warnings
if not sys.warnoptions:
    warnings.simplefilter('ignore')
    
# Don't generate bytecode
sys.dont_write_bytecode = True

In [ ]:
import holoviews as hv
import xarray as xr

from holoviews import opts, dim
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
from holoviews.streams import Stream, param
from holoviews import streams
import geoviews as gv
import geoviews.feature as gf
from geoviews import tile_sources as gvts

import panel as pn
import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cf

hv.notebook_extension('bokeh')
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d'
pn.extension()

In [ ]:
# Add the utils directory to the search path
UTILS_DIR = Path('../utils').absolute()
assert UTILS_DIR.exists()
if str(UTILS_DIR) not in sys.path:
    sys.path.insert(0, str(UTILS_DIR))
    print(f"Added {str(UTILS_DIR)} to sys.path")

# pp(sys.path)
    

In [ ]:
from utils import get_mro as mro, nprint
import utils as u
from vector import Vector as vec

### Set visualization options

In [ ]:
H, W = 500,500

In [ ]:
%opts Image [colorbar=True, active_tools=['wheel_zoom'], tools=['hover']] Curve [tools=['hover'], active_tools=['wheel_zoom']] RGB [active_tools=['wheel_zoom'], tools=['hover']]

In [ ]:
img_opts = opts.Image(height=H, width=W)
vfield_opts = opts.VectorField(width=W, height=H, color='Magnitude',
#                                magnitude=dim('Magnitude').norm()*0.2,
                               pivot='tip',
                               rescale_lengths=True)
curve_opts = opts.Points(size=5,width=W, height=H, padding=0.1, 
#                             xlim=(-10,10), ylim=(-10,10),
#                         color=dim('p')*256-50
                        )
contour_opts = opts.Contours(width=W, height=H, 
                             colorbar=True, 
                             tools=['hover'])

In [ ]:
# Grab registered bokeh renderer
print("Currently available renderers: ", *hv.Store.renderers.keys())
renderer = hv.renderer('bokeh')

In [ ]:
import streamz
import streamz.dataframe

To deinf a curve on a plane (ie. planary curve) we need
- parameter, eg. p $\in [0,1]$
- two functions $x(p)$ and $y(p)$, which define the coordinate of the point $C(p)$ in x and y axis, respectively

## Signed Distance Function 
### For a circle

- $\phi(x,y)$ is the distance between point $p = (x,y)$ and the curve. Here the curve we want to represent is a unit circle
- $\phi(x,y)$ is Dist(origin, $p$) - Dist(origin, a point on the unit circle touched by a stright ray passing through the origin and $p$). The second term is always 1 for a unit circle. So we get the $\phi$ as following:

$ \phi(x,y) = \sqrt{(x^2+y^2)} - 1 $

Now let's visualize it using holoviews.

In [ ]:
def phi(x,y):
    """Signed Distance function for a unit circle"""
    return np.sqrt(x**2 + y**2) - 1


In [ ]:
n = 100
xs = np.linspace(-1.5,1.5,num=n)
ys = np.linspace(-1.5,1.5,num=n)[::-1] #flipped because I'm interested in the cartesian coordinate system's view
# nprint(xs,ys)

In [ ]:
X,Y = np.meshgrid(xs,ys)
Z = phi(X,Y)

In [ ]:
# See hv.Image?
cmap = ['twilight', 'RdBu','Spectral']
img = hv.Image((xs,ys,Z)).opts(img_opts).opts(cmap=cmap[1])
contour = hv.operation.contours(img, levels=1).opts(contour_opts).opts(cmap='gray') #love this level=0:D
img * contour
#or, equivalently hv.Image(Z, bounds=(xs.min(), ys.min(), xs.max(), ys.max())).opts(img_opts)#.opts(cmap='twilight')

In [ ]:
kernel = np.array([[-0.5,0,0.5]])
corr_opts = dict(mode='same')
Zx = correlate2d(Z,kernel, **corr_opts)
Zy= correlate2d(Z, kernel.T, **corr_opts)

(
    hv.Image((xs,ys,Zx)) + hv.Image((xs,ys,Zy))
).opts(img_opts);

In [ ]:
def img_contour(xs, ys, Z, cmap=None, show_contour=True, levels=1):
    if cmap is None:
        cmap=dict(img_cmap='RdBu', contour_cmap='gray')
    img = hv.Image((xs,ys,Z)).opts(img_opts).opts(cmap=cmap['img_cmap'])
    contour = hv.operation.contours(img, levels=levels).opts(contour_opts).opts(cmap=cmap['contour_cmap']) #love this level=0:D
    if show_contour:
        return img*contour
    return img

def UV2angMag(U,V):
    """
    U,V (MxN np.ndarray): encodes X,Y coordinate grids respectively
    Returns:
    - angle, mag: tuple of MxN np.ndarray that encode ang (or mag) for the grid space
    That means, angle[j][i] at (X[j][i],Y[j][i]) location
    """
    mag = np.sqrt(U**2 + V**2)
    angle = (np.pi/2.) - np.arctan2(U/mag, V/mag)
#     angle =  np.arctan2(V,U)


    return (angle, mag)
    

In [ ]:
# img_contour(xs,ys,Zx)+ img_contour(xs,ys,Zy)#,show_contour=False, levels=5)
grad_angle, grad_mag = UV2angMag(Zx,Zy)
gradfield = hv.VectorField( (X,Y,grad_angle, grad_mag) ).opts(vfield_opts)#.opts(height=1000, width=1000)

# img.opts(height=1000,width=1000) * contour * gradfield


---
Modified: Jul 19, 2019
## Curve Evolution 
- [ ] Finite Difference method on parametric equations
- [ ] Levelset functions
    - 2D signed distance functions: [src](https://is.gd/t7p5mk)
- [ ] Active contour on satellite images
- [ ] Agent-based modelling with specified rules
    - satellite image segmentation (~ clustering based on local features)

In [ ]:
import imageio as iio

In [ ]:
test_img = iio.imread('../data/test/gradient.jpg')
bounds
print(test_img.shape)

In [ ]:
test_x, test_y = np.meshgrid(range(test_img.shape[1]), range(test_img.shape[0]))
test_x.shape, test_y.shape
# hv.RGB(test_img. 

In [ ]:
r,g,b = np.dsplit(test_img, test_img.shape[-1])
bounds = (0, 0, test_x.shape[1], test_x.shape[0])
(
    hv.Image( r.squeeze(), bounds=bounds)
    +  hv.Image( g.squeeze(), bounds=bounds)
    +  hv.Image( b.squeeze(), bounds=bounds)
).cols(1).opts(
    opts.Image(height=test_x.shape[0], width=test_x.shape[1])
)

In [ ]:
import vec2 as vec2 

In [ ]:
del vec2

In [ ]:
type(vec2)

In [ ]:
class CurveSimulator(param.Parameterized):

    n_steps = param.Integer(label='Number of simulation steps', default=100)
    p = param.ObjectSelector(label='p', default=0., objects=np.linspace(0,1,num=n_steps.default))
    reset = param.Action(lambda x: x.reset_handler(), doc="Click to clear the buffer and reset p")
    t_interval = param.Number(label='t_interval', doc='Time interval between plotting two points',
                              softbounds=(0., 5.),
                              default=0.)

    
    ################################################################################
    # Constant class properties
    ################################################################################
    H,W = 500,500
    curve_opts = opts.Points(size=5,width=W, height=H, 
                             xlim=(-1,1), ylim=(-1,1),
                             color=dim('p')*256-50,
                             tools=['hover']
                            )
    xopts = opts.Points('XCoord', width=W, height=H, size=5, xlim=(0,1), 
                        padding=0.1, invert_axes=True, invert_yaxis=True)
    yopts = opts.Points('YCoord', width=W, height=H, size=5, xlim=(0,1), padding=0.1, invert_xaxis=True)
    
    
    ################################################################################
    # Parameter Dependencies
    ################################################################################    
    @param.depends('n_steps', watch=True)
    def _update_p(self):
        self.count['p'] += 1
        self.param['p'].objects = np.linspace(0,1,num=self.n_steps)
        print('updated p with new number of simulation steps: ', self.n_steps)
    
    @param.depends('p', watch=True)
    def send_point(self):
        point = pd.DataFrame([(self.p, *self.cfunc(self.p))], columns=['p','x','y'])
        self.data_src.emit(point)
        time.sleep(self.t_interval)
        
    def reset_handler(self):
        self.set_param(p=0.0)
        self.dfstream.clear()

        
    ################################################################################
    # Initialization
    ################################################################################
    def __init__(self, cfunc, n_steps=100, **kwargs):
        """
        Args:
        - cfunc (function): given an input of a float p in [0,1], returns (x,y), a 
        tuple of x and y coords
        
        - n_steps (int): number of simulation steps along the range of [0,1] for 
        the parameter, p
        """
        super().__init__(**kwargs) # this is super important
        self.cfunc = cfunc 
        self.n_steps = n_steps
        
        self.example = pd.DataFrame({'p': [], 'x':[], 'y':[]})
        self.data_src = streamz.dataframe.DataFrame(example=self.example)
        self.dfstream = Buffer(self.data_src, length=min(self.n_steps, 100), index=False)
        self.set_dmap_curve()
        self.set_dmap_x()
        self.set_dmap_y()
        self.overlay = (self.dmap_curve + self.dmap_y + self.dmap_x).cols(2)
    

    def set_dmap_curve(self):
        dmap_curve = hv.DynamicMap(
            lambda data: hv.Points(data, kdims=['x','y'], group='Curve'),
            streams=[self.dfstream])#.opts(color='p')
        self.dmap_curve = dmap_curve.opts(self.curve_opts)
        
    def set_dmap_x(self):
        dmap_x = hv.DynamicMap(
            lambda data: hv.Points( data, kdims=['p','x'], group='XCoord'),
            streams=[self.dfstream]).opts(color='p')
        self.dmap_x = dmap_x.opts(self.xopts)
        
    def set_dmap_y(self):
        dmap_y = hv.DynamicMap(
            lambda data: hv.Points( data, kdims=['p','y'], group='YCoord'),
            streams=[self.dfstream]).opts(color='p')
        self.dmap_y = dmap_y.opts(self.yopts)
    
    
    ################################################################################
    # Display DynammicMaps
    ################################################################################ 
    def viewable(self):
        return self.overlay
        


### Define curve function

In [ ]:
xfunc = lambda p: np.sin(2*np.pi*p)
yfunc = lambda p: np.cos(2*np.pi*p)

In [ ]:
# Try different functions
xfunc = lambda p: np.sin(2*np.pi*p)**2
yfunc = lambda p: np.cos(2*np.pi*p)

In [ ]:
xfunc = lambda p: np.sin(2*np.pi*p)**10
yfunc = lambda p: np.cos(2*np.pi*p)

In [ ]:
# Alternating along a straight line 
xfunc = lambda p: np.sin(2*np.pi*p)**2
yfunc = lambda p: np.cos(2*np.pi*p)**2

In [ ]:
# Doesn't have to choose a periodic function
xfunc = lambda p: np.log(p)
yfunc = lambda p: p

In [ ]:
# Something happens at p=0.5
xfunc = lambda p: np.sin(2*np.pi*p**2)*p**3
yfunc = lambda p: np.sin(np.pi*p**0.5)

### Create the simulator for the curve

In [ ]:
cfunc = lambda p: (xfunc(p), yfunc(p))
c = CurveSimulator(cfunc)

### Show the simulator

In [ ]:
pn.Row(
    pn.Param(c.param, width=500, widgets={
        'p': pn.widgets.DiscretePlayer,
        'reset': pn.widgets.Button(name=c.param['reset'].label),
        't_interval': pn.widgets.FloatSlider
    }),
    pn.panel(c.viewable())
)

In [ ]:
c.dfstream.data